# feature selection and model training

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score

In [4]:
import os

os.chdir("../")
print(os.getcwd())

c:\Users\valen\Desktop\etl_workshop003


In [ ]:
df_model = pd.read_csv('data/merged_data.csv')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
import joblib

In [6]:
# 🔢 1. CARGAR Y PREPARAR DATOS
df = pd.read_csv('data/merged_data.csv')  # Cambia por tu archivo real

# Asegúrate de que 'country' esté en el DataFrame original
df_codificado = pd.get_dummies(df, columns=["country"], drop_first=True)

# Puedes verificar las nuevas columnas así:
print(df_codificado.columns)

# Define tus X e y nuevamente
X = df_codificado.drop(columns=["happiness_score", "region", 'perceptions_of_corruption', 'generosity', 'year'])
y = df_codificado["happiness_score"]
# 📚 2. DIVIDIR TRAIN Y TEST
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔁 3. DEFINIR MODELOS A PROBAR
modelos = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# 🧪 4. ENTRENAR, EVALUAR Y COMPARAR
resultados = []

for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    resultados.append((nombre, r2, rmse))
    print(f"{nombre}: R² = {r2:.3f}, RMSE = {rmse:.3f}")

# 🏆 5. ELEGIR EL MEJOR MODELO
mejor_modelo = max(resultados, key=lambda x: x[1])  # el que tenga mayor R²
nombre_mejor, r2_mejor, rmse_mejor = mejor_modelo

print(f"\n✅ Mejor modelo: {nombre_mejor} (R² = {r2_mejor:.3f}, RMSE = {rmse_mejor:.3f})")

# 🔄 ENTRENARLO CON TODO EL DATASET
modelo_final = modelos[nombre_mejor]
modelo_final.fit(X, y)

# 💾 6. GUARDAR EL MODELO ENTRENADO
'''joblib.dump(modelo_final, f"modelo_{nombre_mejor.replace(' ', '_').lower()}.pkl")
print(f"Modelo guardado como modelo_{nombre_mejor.replace(' ', '_').lower()}.pkl")
'''

Index(['region', 'happiness_score', 'gdp_per_capita', 'social_support',
       'healthy_life_expectancy', 'freedom', 'perceptions_of_corruption',
       'generosity', 'year', 'country_albania',
       ...
       'country_united arab emirates', 'country_united kingdom',
       'country_united states', 'country_uruguay', 'country_uzbekistan',
       'country_venezuela', 'country_vietnam', 'country_yemen',
       'country_zambia', 'country_zimbabwe'],
      dtype='object', length=171)
Linear Regression: R² = 0.952, RMSE = 0.245
Ridge: R² = 0.930, RMSE = 0.296
Lasso: R² = 0.613, RMSE = 0.694
Random Forest: R² = 0.859, RMSE = 0.419
Gradient Boosting: R² = 0.830, RMSE = 0.459

✅ Mejor modelo: Linear Regression (R² = 0.952, RMSE = 0.245)


'joblib.dump(modelo_final, f"modelo_{nombre_mejor.replace(\' \', \'_\').lower()}.pkl")\nprint(f"Modelo guardado como modelo_{nombre_mejor.replace(\' \', \'_\').lower()}.pkl")\n'

In [ ]:
X = df_model.drop(['happiness_score', 'region', 'freedom', 'perceptions_of_corruption', 'generosity', 'year'], axis=1)
y = df_model['happiness_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Data Preprocessing

***One-Hot Encoding for Categorical Features***

Apply One-Hot Encoding to categorical features to convert them into numerical format.

In [ ]:
categorical_features = ['country']
preprocessor = ColumnTransformer(
	transformers=[
		('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
	],
	remainder='passthrough'
)

X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded = preprocessor.transform(X_test)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)